# Predicting lexical errors:

In [2]:
import os, re, random, json

In [3]:
import pandas as pd

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.data import Field, LabelField, RawField, BucketIterator, TabularDataset, Iterator
from torchtext.vocab import Vectors
from sklearn.metrics import accuracy_score, f1_score
from spellchecker import SpellChecker

from tqdm import tqdm, tqdm_notebook

from nltk.stem import WordNetLemmatizer
from classifier import *

tqdm.pandas()

c:\users\k1l77\python_envs\torch_env\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
def dump_vocab(vocab, dir_name):
  os.makedirs(dir_name, exist_ok=True)
  with open(os.path.join(dir_name, 'freqs.json'), 'w', encoding='utf-8') as foutp:
    json.dump(vocab.freqs, foutp, ensure_ascii=False, indent=2)
  with open(os.path.join(dir_name, 'itos.json'), 'w', encoding='utf-8') as foutp:
    json.dump(vocab.itos, foutp, ensure_ascii=False, indent=2)
  with open(os.path.join(dir_name, 'stoi.json'), 'w', encoding='utf-8') as foutp:
    json.dump(vocab.stoi, foutp, ensure_ascii=False, indent=2)

def load_vocab(dir_name):
  freqs_path = os.path.join(dir_name, 'freqs.json')
  itos_path = os.path.join(dir_name, 'itos.json')
  stoi_path = os.path.join(dir_name, 'stoi.json')

  with open(freqs_path, 'r', encoding='utf-8') as finp:
    freqs = Counter(json.load(finp))
  with open(itos_path, 'r', encoding='utf-8') as finp:
    itos = json.load(finp)
  with open(stoi_path, 'r', encoding='utf-8') as finp:
    stoi = json.load(finp)
  
  vocab = Vocab(freqs)
  vocab.itos = itos
  vocab.stoi = stoi

  return vocab

In [5]:
def remove_err_span(string):
    match = re.search('<b>.*?</b>', string)
    return string[:match.start()], string[match.end():]

## Подготовка датасета:

Попробуем сначала существительные, так как их больше всего. Бейзлайн по accuracy - 0,115375

In [15]:
df = pd.read_excel("Nouns.xlsx", index_col='id')

In [16]:
df.head()

,Sentence,Error type,Wrong answer,Right answer,error POS,correction POS,Right answer lemma,Wrong answer lemma
id,,,,,,,,
156,In Africa from 2000 to 2012 there was a decre...,lex_item_choice,movement,tendency,NN,NN,tendency,movement
174,But they forget about the huge <b>causes</b> ...,lex_item_choice,causes,consequences,NNS,NNS,consequence,cause
203,There is the opinion that flights do <b>harm<...,lex_item_choice,harm,influence,NN,NN,influence,harm
221,The percentage of boys in the same <b>ratio</...,lex_item_choice,ratio,region,NN,NN,region,ratio
227,"And the last region, which had the third plac...",lex_item_choice,structure,proportion,NN,NN,proportion,structure


In [17]:
df = df.drop(['Error type','error POS', 'correction POS'], axis=1)

In [20]:
df[['Left','Right']] = df.apply(lambda x: remove_err_span(x['Sentence']),
                              axis=1, result_type="expand")
df = df.drop(['Sentence'], axis=1)

In [21]:
df = df[['Left','Right','Wrong answer lemma','Right answer lemma']]

In [22]:
df.head()

,Left,Right,Wrong answer lemma,Right answer lemma
id,,,,
156,In Africa from 2000 to 2012 there was a decre...,from 23.,movement,tendency
174,But they forget about the huge,of such freedom - air is being polluted and c...,cause,consequence
203,There is the opinion that flights do,to air pollution and global warming and that ...,harm,influence
221,The percentage of boys in the same,was made up approximately 20 millions in 2000...,ratio,region
227,"And the last region, which had the third plac...",of men and women was approximately equal (50 ...,structure,proportion


In [23]:
df.shape

(4195, 4)

In [24]:
df['Wrong answer lemma'].value_counts().quantile(0.7)

3.0

In [25]:
sum([i for i in df['Wrong answer lemma'].value_counts() if i > 5])

2514

In [30]:
df['Rare'] = df.apply(lambda x: df['Wrong answer lemma'].value_counts()[x['Wrong answer lemma']] < 5 and df['Right answer lemma'].value_counts()[x['Right answer lemma']] < 5,
                      axis=1)

In [31]:
df = df[df['Rare']==False]

In [32]:
df.shape

(3343, 5)

In [33]:
len(df['Wrong answer lemma'].value_counts())+len(df['Right answer lemma'].value_counts())

876

In [34]:
df.to_csv("Nouns_dataset.csv")

Потом глаголы:

In [20]:
df = pd.read_excel("Verbs.xlsx", index_col='id')

In [21]:
df.head()

,Sentence,Error type,Wrong answer,Right answer,error POS,correction POS,Right answer lemma,Wrong answer lemma
id,,,,,,,,
22,Alprin offered to use the First Amendment in ...,lex_item_choice,apply,refer,VB,VB,refer,apply
33,Considerable social and financial resources a...,lex_item_choice,sent,allocated,VBN,VBN,allocate,send
44,But the main thing is just to start doing som...,lex_item_choice,come,risen,VBN,VBN,rise,come
114,In contrast indicators of South Asia in 2012 ...,lex_item_choice,have,show,VBP,VBP,show,have
167,Many people <b>consider</b> that governments ...,lex_item_choice,consider,believe,VBP,VBP,believe,consider


In [22]:
df = df.drop(['Error type'], axis=1)

In [23]:
df[['Left','Right']] = df.apply(lambda x: remove_err_span(x['Sentence']),
                              axis=1, result_type="expand")
df = df.drop(['Sentence'], axis=1)

In [24]:
df = df[['Left','Right','Wrong answer lemma','Right answer lemma','error POS','correction POS']]

In [25]:
df.head()

,Left,Right,Wrong answer lemma,Right answer lemma,error POS,correction POS
id,,,,,,
22,Alprin offered to use the First Amendment in ...,to the First Amendment in a slip-and-fall cas...,apply,refer,VB,VB
33,Considerable social and financial resources are,to solving this problem.,send,allocate,VBN,VBN
44,But the main thing is just to start doing som...,to the unthinkable extent.,come,rise,VBN,VBN
114,In contrast indicators of South Asia in 2012,the smallest number of children without acces...,have,show,VBP,VBP
167,Many people,that governments should support the decrease ...,consider,believe,VBP,VBP


In [26]:
df.shape

(3181, 6)

In [27]:
checker = SpellChecker()

Исправим опечатки:

In [28]:
df['Wrong answer lemma'] = df['Wrong answer lemma'].apply(lambda x: checker.correction(x))

In [30]:
df['Wrong answer lemma'] = df['Wrong answer lemma'].apply(lambda x: x.lower())

In [31]:
df['Wrong answer lemma'].value_counts().quantile(0.7)

3.0

In [32]:
sum([i for i in df['Wrong answer lemma'].value_counts() if i > 2])

2629

In [33]:
df['Rare'] = df.apply(lambda x: df['Wrong answer lemma'].value_counts()[x['Wrong answer lemma']] < 3 and df['Right answer lemma'].value_counts()[x['Right answer lemma']] < 3,
                      axis=1)

In [34]:
df = df[df['Rare']==False]

In [35]:
df.shape

(3022, 7)

In [36]:
df['Wrong answer lemma'].unique()

array(['apply', 'send', 'come', 'have', 'consider', 'surround', 'accept',
       'enter', 'fix', 'go', 'rise', 'produce', 'put', 'diction',
       'access', 'increase', 'indicate', 'do', 'decide', 'note', 'show',
       'be', 'take', 'learn', 'spend', 'illustrate', 'get', 'make',
       'digest', 'reduce', 'cancel', 'travel', 'stand', 'speak',
       'exhaust', 'achieve', 'bring', 'understand', 'solute', 'tell',
       'embroil', 'establish', 'turn', 'present', 'develop', 'content',
       'base', 'stay', 'relay', 'touch', 'play', 'belong', 'use', 'work',
       'current', 'suppose', 'remind', 'say', 'combine', 'hold',
       'perceive', 'happen', 'mark', 'decrease', 'return', 'conclude',
       'leave', 'stimulate', 'deliver', 'witness', 'ruin', 'describe',
       'realize', 'admire', 'consist', 'collect', 'waste', 'state',
       'pass', 'habituate', 'give', 'suggest', 'notice', 'aim',
       'discover', 'slow', 'invent', 'write', 'create', 'comprise',
       'introduce', 'rat', 'rel

In [37]:
df.to_csv("Verbs_dataset.csv")

Потом прилагательные:

In [6]:
df = pd.read_excel("Adjectives.xlsx", index_col='id')

In [7]:
df.head()

,Sentence,Error type,Wrong answer,Right answer,error POS,correction POS,Right answer lemma,Wrong answer lemma
id,,,,,,,,
9,Alicia and her colleague Kalinda thought that...,lex_item_choice,usual,simple,JJ,JJ,simple,usual
31,But after a break in a trial the prosecutor s...,lex_item_choice,indifferent,negligent,JJ,JJ,negligent,indifferent
155,In Africa from 2000 to 2012 there was a decre...,lex_item_choice,common,same,JJ,JJ,same,common
171,That is why laws should forbid the huge numbe...,lex_item_choice,definite,certain,JJ,JJ,certain,definite
249,The most <b>gradual</b> decrease was outlined...,lex_item_choice,gradual,dramatic,JJ,JJ,dramatic,gradual


In [8]:
df = df.drop(['Error type','error POS', 'correction POS'], axis=1)

In [9]:
df[['Left','Right']] = df.apply(lambda x: remove_err_span(x['Sentence']),
                              axis=1, result_type="expand")
df = df.drop(['Sentence'], axis=1)

In [10]:
df = df[['Left','Right','Wrong answer lemma','Right answer lemma']]

In [11]:
df.head()

,Left,Right,Wrong answer lemma,Right answer lemma
id,,,,
9,Alicia and her colleague Kalinda thought that...,house with her Jewish husband.,usual,simple
31,But after a break in a trial the prosecutor s...,about the defective eruv wire.,indifferent,negligent
155,In Africa from 2000 to 2012 there was a decre...,tendency from 23.,common,same
171,That is why laws should forbid the huge numbe...,number of flights in each period to combine t...,definite,certain
249,The most,decrease was outlined in south Asia and reach...,gradual,dramatic


In [12]:
df.to_csv("Adjectives_dataset.csv")

## Работа с датасетом:

In [13]:
vectors = Vectors('gensim_models/WebVectores 0/model.txt')

In [14]:
TOKENS = Field(lower=True, tokenize=nltk.word_tokenize)
ANSWER = LabelField(dtype=tt.int64, use_vocab=True, unk_token='<unk>')
ID = RawField()

In [15]:
lexics_dataset = TabularDataset("Adjectives_dataset.csv", format='csv',
                                fields=[('context_id',ID), ('left',TOKENS), ('right', TOKENS),
                                        ('wrong_item',ANSWER), ('right_item',ANSWER),
                                        (None,None)],
                                skip_header=True)

In [16]:
random.seed(42)

In [17]:
train_set, valid_set = lexics_dataset.split(0.9, random_state=random.getstate())

In [18]:
TOKENS.build_vocab(train_set)
ANSWER.build_vocab(train_set, valid_set, vectors=vectors)

unk_answer_id = len(ANSWER.vocab.itos)
ANSWER.vocab.itos.append('<unk>')
ANSWER.vocab.stoi['<unk>'] = unk_answer_id
ANSWER.vocab.freqs['<unk>'] = 0
ANSWER.vocab.vectors = tt.cat((ANSWER.vocab.vectors, tt.zeros(1, 300, dtype=tt.float32)), 0)

In [19]:
ANSWER.vocab.vectors.shape

torch.Size([693, 300])

In [20]:
len(ANSWER.vocab.itos)

693

In [21]:
len(TOKENS.vocab.itos)

2714

In [29]:
def calculate_accuracy_and_f1(true, pred):
    pred = pred.data.cpu()
    true = true.data.cpu()
    accuracy = accuracy_score(true, pred)
    f1 = f1_score(true, pred, average='weighted')
    return accuracy, f1

def invert_seq_batch(batch):
    ## Solution from https://discuss.pytorch.org/t/how-to-use-a-lstm-in-a-reversed-direction/14389
    inv_idx = tt.arange(batch.size(1)-1, -1, -1).long()
    return batch.index_select(1, inv_idx)


class Batch:
    def __init__(self, batch, device):
        self.left = batch.left.to(device)
        self.right = invert_seq_batch(batch.right).to(device)
        self.wrong_item = batch.wrong_item.to(device)
        self.right_item = batch.right_item.to(device)

In [30]:
def train(model, train_iterator, optimizer, criterion, device,
          scheduler, n_epochs, penalty_multiplier=1.1):
    
    for epoch in range(n_epochs):
        train_loss, train_acc, train_f1 = train_epoch(model, train_iterator, optimizer, criterion, device, epoch,
                                                     penalty_multiplier)
        if scheduler:
            scheduler.step(train_loss)
        print(f"Training loss: {round(train_loss,5)} accuracy: {round(train_acc,4)} f1: {round(train_f1,4)}")

def train_epoch(model, train_iterator, optimizer, criterion, device, n_epoch,
                penalty_multiplier=1):
    losses = []
    model.train()
    n_batches = len(train_iterator)
    iterator = tqdm_notebook(train_iterator, total=n_batches, desc=f"Training epoch {n_epoch}", leave=True)
    
    for i, batch in enumerate(iterator):
        optimizer.zero_grad()
        
        batch = Batch(batch, device)
        pred = model(batch)
        ## Criterion should have param "reduce" set to None
        loss = criterion(pred, batch.wrong_item)
        
        if penalty_multiplier != 1:
            ## check if predicted value equals right tense:
            pen_vec = pred.argmax(dim=1) == batch.right_item
            ## form penalty vector:
            penalty_multiplier = tt.Tensor([penalty_multiplier]).to(device)
            pen_vec = pen_vec.type(tt.float).to(device)
            pen_vec = (pen_vec+1)**tt.log2(penalty_multiplier)
            ## punish if it equals:
            loss = loss * pen_vec
        loss = loss.mean()
        loss.backward()
        optimizer.step()
        
        curr_loss = loss.data.cpu().detach().item()
        iterator.set_postfix(loss=str(round(curr_loss,5)))
        acc, f1 = calculate_accuracy_and_f1(batch.wrong_item, pred.argmax(dim=1))
        losses.append((curr_loss, acc, f1))
    return np.mean(losses, axis=0)

def get_k_predicted_items(model, iterator, device, inv_labels, k=4):
    output = []
    model.eval()
    n_batches = len(iterator)
    batch_iter = tqdm_notebook(iterator, total=n_batches)
    with tt.no_grad():
        for batch in batch_iter:
            batch_gpu = Batch(batch, device)
            pred = F.softmax(model(batch_gpu).data.cpu())
            top_k = pred.topk(k=k, dim=1)
            top_probas, top_ids = top_k.values, top_k.indices
            tenses = [[inv_labels[int(i)] for i in text] for text in top_ids]
            output.append((batch.context_id, tenses, top_probas))
    return output

In [31]:
class W2VErrorModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden1_size, hidden2_size, output_size, vectors,
                 dropout1_rate=0.05, dropout2_rate=0.1, dropout3_rate=0.2, activation=nn.ReLU()):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)#.from_pretrained(vectors, freeze=False)
        self.frozen_embedding = nn.Embedding(output_size, embed_size).from_pretrained(vectors, freeze=False)
        
        self.dropout1 = nn.Dropout(p=dropout1_rate)
        
        self.lstm1 = nn.LSTM(input_size=embed_size,
                             hidden_size=hidden1_size,
                             batch_first=False,
                             bidirectional=False)
        self.lstm2 = nn.LSTM(input_size=embed_size,
                             hidden_size=hidden1_size,
                             batch_first=False,
                             bidirectional=False)
        
        self.dropout2 = nn.Dropout(p=dropout2_rate)
        self.fc1 = nn.Linear(hidden1_size*2+embed_size, hidden2_size)
        self.activation = activation
        self.dropout3 = nn.Dropout(p=dropout3_rate)
        self.fc2 = nn.Linear(hidden2_size, output_size)
    
    def forward(self, batch):
        ## Run forward-LSTM on left context of error:
        left = self.embedding(batch.left)
        left = self.dropout1(left)
        left, _ = self.lstm1(left)
        left = left[-1]
        
        ## Run backward-LSTM on Left context of error:
        right = self.embedding(batch.right)
        right = self.dropout1(right)
        right, _ = self.lstm2(right)
        right = right[-1]
        
        correction = self.frozen_embedding(batch.right_item)
        
        ## Concatenate:
        x = tt.cat([left,right,correction],dim=1)
        x = self.dropout2(x)
        
        ## Apply fully connected layers:
        x = self.dropout3(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x

In [32]:
device = tt.device('cuda')

tt.cuda.empty_cache()

batch_size = 128

train_iter, val_iter = BucketIterator.splits((train_set, valid_set),
                                                             batch_sizes=(batch_size,batch_size,batch_size),
                                                             shuffle=False, sort_key=lambda x:len(x.left)+len(x.right))

model = W2VErrorModel(vocab_size=len(TOKENS.vocab.itos),embed_size=300,
                      vectors=ANSWER.vocab.vectors,
                hidden1_size=300, hidden2_size=200, output_size=len(ANSWER.vocab.itos))
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = tt.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)

In [33]:
history = train(model, train_iter, optimizer, criterion, device, scheduler, n_epochs=500, penalty_multiplier=2.5)

c:\users\k1l77\python_envs\torch_env\lib\site-packages\ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


c:\users\k1l77\python_envs\torch_env\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\k1l77\python_envs\torch_env\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



Training loss: 6.60432 accuracy: 0.018 f1: 0.0054



Training loss: 6.67237 accuracy: 0.0707 f1: 0.0102



Training loss: 6.56604 accuracy: 0.0707 f1: 0.01


...


Training loss: 0.03334 accuracy: 0.9993 f1: 0.999



Training loss: 0.03199 accuracy: 0.9979 f1: 0.9976



Training loss: 0.03246 accuracy: 0.9993 f1: 0.9993



Training loss: 0.03123 accuracy: 0.9972 f1: 0.9972



Training loss: 0.02942 accuracy: 0.9986 f1: 0.9983


In [34]:
model.frozen_embedding

Embedding(693, 300)

In [35]:
tt.save(model.state_dict(), 'adj_lex_model')
dump_vocab(TOKENS.vocab, 'adj_question_vocab')
dump_vocab(ANSWER.vocab, 'adj_answer_vocab')

In [36]:
val_loss, val_acc, val_f1 = val(model, val_iter, criterion, device, penalty_multiplier=2.5)
val_loss, val_acc, val_f1

(3.855257749557495, 0.614514802631579, 0.6200814536340852)

In [37]:
predictions = get_k_predicted_items(model,train_iter,device, inv_labels=ANSWER.vocab.itos, k=6)

all_predictions = dict()

for batch in tqdm_notebook(predictions, total=len(predictions)):
    id_batch, proba_batch, prep_batch = batch
    for index, preps, probas in zip(id_batch, proba_batch, prep_batch):
        all_predictions[index] = list(zip(preps, probas))

data = pd.read_csv("Verbs_dataset.csv", index_col='id')

def add_option(row, skip_list=['of','is']):
    if str(row.name) in all_predictions:
        row_predictions = all_predictions[str(row.name)]
        options = [i[0] for i in sorted(row_predictions,
                                    key=lambda x: -x[1])]
        options = [i for i in options if i!=row['Wrong answer lemma'] and i!=row['Right answer lemma'] and i not in skip_list]
        return options[0], options[1]
    else:
        return '', ''

data[['Option 1','Option 2']] = data.apply(lambda x: add_option(x),
                                          axis=1, result_type="expand")
data = data[(data['Option 1']!='')&(data['Option 2']!='')]
data = data.drop(['Rare'], axis=1)

data.to_excel("Predicted adjectives train.xlsx")

C:\Users\k1l77\Desktop\Diploma\classifier.py:199: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.softmax(model(batch_gpu).data.cpu())


c:\users\k1l77\python_envs\torch_env\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [38]:
predictions = get_k_predicted_items(model, val_iter,device, inv_labels=ANSWER.vocab.itos, k=6)

all_predictions = dict()

for batch in tqdm_notebook(predictions, total=len(predictions)):
    id_batch, proba_batch, prep_batch = batch
    for index, preps, probas in zip(id_batch, proba_batch, prep_batch):
        all_predictions[index] = list(zip(preps, probas))

data = pd.read_csv("Verbs_dataset.csv", index_col='id')

def add_option(row, skip_list=['of','is']):
    if str(row.name) in all_predictions:
        row_predictions = all_predictions[str(row.name)]
        options = [i[0] for i in sorted(row_predictions,
                                    key=lambda x: -x[1])]
        options = [i for i in options if i!=row['Wrong answer lemma'] and i!=row['Right answer lemma'] and i not in skip_list]
        return options[0], options[1]
    else:
        return '', ''

data[['Option 1','Option 2']] = data.apply(lambda x: add_option(x),
                                          axis=1, result_type="expand")
data = data[(data['Option 1']!='')&(data['Option 2']!='')]
data = data.drop(['Rare'], axis=1)

data.to_excel("Predicted adjectives test.xlsx")

c:\users\k1l77\python_envs\torch_env\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """
